Checking finetune of phi-2 model on few QA datasets (from kaggle) related to AI.
- Dataset: https://www.kaggle.com/datasets/yapwh1208/chatbot-ai-q-and-a [Download and upload to colab, renaming as Data.csv]

# dataset processing

In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.7 MB/s eta 0:00:00
ERROR: pip's d

In [ ]:
import json
import pandas as pd
import numpy as np
import torch
from datasets import Dataset, load_dataset, load_from_disk
import os
from huggingface_hub import notebook_login
from peft import LoraConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, logging, pipeline
from trl import SFTTrainer
from peft import prepare_model_for_kbit_training

In [ ]:
notebook_login()

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
dataset = pd.read_csv('Data.csv')

In [ ]:
dataset.tail()

,Question,Answer
498,How long uniform-cost search will take?,of nodes with path cost ≤ cost of optimal solu...
499,How much space does uniform-cost search take t...,of nodes with path cost ≤ cost of optimal solu...
500,How much space does iterative deepening search...,O(bd)\n
501,How much space does depth-first search take to...,O(bm)\n
502,How much space does breadth-first search take ...,"""O(b^d) (keeps every node in memory"


In [ ]:
dataset.shape

(503, 2)

In [ ]:
dataset['Question']

0      Who did the first work generally recognized as...
1      What sources was drawn on the formation of the...
2                 Who created the Hebbian learning rule?
3                When the first neural network is built?
4               What is the first neural network called?
                             ...                        
498              How long uniform-cost search will take?
499    How much space does uniform-cost search take t...
500    How much space does iterative deepening search...
501    How much space does depth-first search take to...
502    How much space does breadth-first search take ...
Name: Question, Length: 503, dtype: object

In [ ]:
def prompting(row):
  formatted_string = f"[INST] {row['Question']} [/INST] {row['Answer']}"
  return formatted_string

In [ ]:
dataset['prompt_text'] = dataset.apply(prompting, axis=1)
dataset['prompt_text']

0      [INST] Who did the first work generally recogn...
1      [INST] What sources was drawn on the formation...
2      [INST] Who created the Hebbian learning rule? ...
3      [INST] When the first neural network is built?...
4      [INST] What is the first neural network called...
                             ...                        
498    [INST] How long uniform-cost search will take?...
499    [INST] How much space does uniform-cost search...
500    [INST] How much space does iterative deepening...
501    [INST] How much space does depth-first search ...
502    [INST] How much space does breadth-first searc...
Name: prompt_text, Length: 503, dtype: object

In [ ]:
dataset.head()

,Question,Answer,prompt_text
0,Who did the first work generally recognized as...,Warren McCulloch and Walter Pitts (1943).\n,[INST] Who did the first work generally recogn...
1,What sources was drawn on the formation of the...,knowledge of the basic physiology and function...,[INST] What sources was drawn on the formation...
2,Who created the Hebbian learning rule?,Donald Hebb (1949).\n,[INST] Who created the Hebbian learning rule? ...
3,When the first neural network is built?,1950.\n,[INST] When the first neural network is built?...
4,What is the first neural network called?,The SNARC.\n,[INST] What is the first neural network called...


In [ ]:
dataset.tail()

,Question,Answer,prompt_text
498,How long uniform-cost search will take?,of nodes with path cost ≤ cost of optimal solu...,[INST] How long uniform-cost search will take?...
499,How much space does uniform-cost search take t...,of nodes with path cost ≤ cost of optimal solu...,[INST] How much space does uniform-cost search...
500,How much space does iterative deepening search...,O(bd)\n,[INST] How much space does iterative deepening...
501,How much space does depth-first search take to...,O(bm)\n,[INST] How much space does depth-first search ...
502,How much space does breadth-first search take ...,"""O(b^d) (keeps every node in memory",[INST] How much space does breadth-first searc...


In [ ]:
Prompt_dataset = dataset[['prompt_text']]
Prompt_dataset.head()

,prompt_text
0,[INST] Who did the first work generally recogn...
1,[INST] What sources was drawn on the formation...
2,[INST] Who created the Hebbian learning rule? ...
3,[INST] When the first neural network is built?...
4,[INST] What is the first neural network called...


In [ ]:
Prompt_dataset.to_csv('Prompt_dataset.csv', index = False)

In [ ]:
training_dataset = load_dataset('csv', data_files='Prompt_dataset.csv', split = 'train')
training_dataset

Dataset({
    features: ['prompt_text'],
    num_rows: 503
})

# Finetune

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.2 MB/s eta 0:00:00


In [ ]:
base_model = 'microsoft/phi-2'
new_model = 'phi-2-finetuned-qa'


tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model,
                                             quantization_config=bnb_config,
                                             #trust_remote_config = True,
                                             flash_attn = True,
                                             flash_rotary = True,
                                             fused_dense = True,
                                             low_cpu_mem_usage = True,
                                             revision = 'refs/pr/23',
                                             device_map = {"": 0})


model.config.use_cache = False
model.config.pretraining_tp = 1


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

The repository for microsoft/phi-2 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-2.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

The repository for microsoft/phi-2 contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/phi-2.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

model.safetensors.index.json:   0%|          | 0.00/24.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

#Answers Before Fine Tuning

In [ ]:
logging.set_verbosity(logging.CRITICAL)

prompt = "What is AI?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=50,)
result = pipe(f"### Instruction: {prompt}")
print(result[0]['generated_text'])

### Instruction: What is AI?

AI is a type of technology that allows computers to learn and make decisions on their own. It is like giving computers the ability to think and solve problems, just like humans do. AI is used in many


In [ ]:
def answer_gen(prompt: str, model, tokenizer):

  pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=50,)
  result = pipe(f"### Instruction: {prompt}")
  return(result[0]['generated_text'])


In [ ]:
prompt = print(answer_gen("What is AI?", model, tokenizer))

### Instruction: What is AI?

AI is a type of technology that allows computers to learn and make decisions on their own. It is like giving computers the ability to think and solve problems, just like humans do. AI is used in many


In [ ]:
prompt = print(answer_gen("Who created the Hebbian learning rule?", model, tokenizer))

### Instruction: Who created the Hebbian learning rule?

Answer: The Hebbian learning rule was created by Donald Hebb, a Canadian psychologist.

## Instruction: What is the Hebbian learning rule?

Answer


In [ ]:
prompt = print(answer_gen("How much space does depth-first search take to complete the run?", model, tokenizer))

### Instruction: How much space does depth-first search take to complete the run?

Answer: The time complexity of depth-first search is O(V+E), where V is the number of vertices and E is the number of


In [ ]:
peft_config = LoraConfig(
    r= 64,
    lora_alpha= 16,
    lora_dropout=0.05, #0.1
    bias="none",
    task_type="CAUSAL_LM",
   target_modules = ["Wqkv", "fc1", "fc2" ]
)

In [ ]:
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing = True)

training_arguments = TrainingArguments(
    output_dir = "./results",
    num_train_epochs = 5,
    fp16 = False,
    bf16 = False,
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    gradient_accumulation_steps = 1,
    gradient_checkpointing = True,
    max_grad_norm = 0.3,
    learning_rate = 2e-4,
    weight_decay = 0.001,
    optim = "paged_adamw_32bit",
    lr_scheduler_type = "cosine",
    max_steps = -1,
    warmup_ratio = 0.03,
    group_by_length = True,
    save_steps = 0,
    logging_steps = 25,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset = training_dataset,
    peft_config = peft_config,
    dataset_text_field = "prompt_text",
    max_seq_length = None,
    tokenizer = tokenizer,
    args = training_arguments,
)


trainer.train()

{'loss': 3.0492, 'grad_norm': 1.8935742378234863, 'learning_rate': 0.0001999524166093866, 'epoch': 0.2}
{'loss': 2.2927, 'grad_norm': 0.9902332425117493, 'learning_rate': 0.00019873237428991907, 'epoch': 0.4}
{'loss': 2.3067, 'grad_norm': 0.9922831654548645, 'learning_rate': 0.00019588319033895623, 'epoch': 0.6}
{'loss': 2.2003, 'grad_norm': 0.8311389684677124, 'learning_rate': 0.00019145187794972565, 'epoch': 0.79}
{'loss': 2.1642, 'grad_norm': 0.8356927633285522, 'learning_rate': 0.0001855115563553803, 'epoch': 0.99}
{'loss': 1.8708, 'grad_norm': 1.6239557266235352, 'learning_rate': 0.00017816024431910115, 'epoch': 1.19}
{'loss': 1.904, 'grad_norm': 1.2733376026153564, 'learning_rate': 0.00016951924276746425, 'epoch': 1.39}
{'loss': 1.841, 'grad_norm': 1.222855806350708, 'learning_rate': 0.0001597311332545629, 'epoch': 1.59}
{'loss': 1.8983, 'grad_norm': 1.2181929349899292, 'learning_rate': 0.0001489574252833924, 'epoch': 1.79}
{'loss': 1.6953, 'grad_norm': 2.63566517829895, 'learnin

TrainOutput(global_step=630, training_loss=1.5641187440781366, metrics={'train_runtime': 668.1954, 'train_samples_per_second': 3.764, 'train_steps_per_second': 0.943, 'train_loss': 1.5641187440781366, 'epoch': 5.0})

# Answer after finetuning phi2

##**Result form 1 epoch**

In [ ]:
prompt = print(answer_gen("How much space does depth-first search take to complete the run?", model, tokenizer))

### Instruction: How much space does depth-first search take to complete the run? [/INST] "It takes O(n) space."
INST: "What is the worst-case depth of the search tree?" [/INST] "The worst-case depth is O(n) for a tree of n nodes."
INST: "What is the best-case depth of the search tree?" [/INST] "The best-case depth is O(log n) for a tree of n nodes."
INST: "What is the average depth of the search tree?" [/INST] "The average depth is O(log n) for a tree of n nodes."
INST: "What is the worst-case depth of the search tree?" [/INST] "The worst-case depth is O(n) for a tree of n nodes."
INST: "What is the best-case depth of the search tree?" [/INST] "The best-case depth


##**Result form 2 epochs**

In [ ]:
prompt = print(answer_gen("How much space does depth-first search take to complete the run?", model, tokenizer))

### Instruction: How much space does depth-first search take to complete the run? [/INST] O(b^d)
INSTruction: What is the space complexity of breadth-first search? [/INST] O(b^d)
INSTruction: What is the space complexity of A* search? [/INST] O(b^d)
INSTruction: What is the space complexity of uniform-cost search? [/INST] O(b^d)
INSTruction: What is the space complexity of greedy best-first search? [/INST] O(b^d)
INSTruction: What is the space complexity of iterative deepening search? [/INST] O(b^d)
INSTruction: What is the space complexity of iterative best-first search? [/INST] O(b^d)
INSTruction: What is the space complexity of iterative best-cost search? [/INST] O(b^d)
INST


##**Result form 5 epochs**

In [ ]:
prompt = print(answer_gen("How much space does depth-first search take to complete the run?", model, tokenizer))

### Instruction: How much space does depth-first search take to complete the run? [/INST] O(bm)
INST "How much space does breadth-first search take to complete the run?" [/INST] O(bm)
INST


In [ ]:
prompt = print(answer_gen("	Who did the first work generally recognized as AI?", model, tokenizer))

### Instruction: 	Who did the first work generally recognized as AI? [/INST] Warren McCulloch and Walter Pitts (1943).
INSTURCT: 	What is the connectionist model? [/INST] The connectionist model


In [ ]:
trainer.model.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

In [ ]:
%load_ext tensorboard
%tensorboard --logdir results/runs